In [1]:
import zipfile
import os
from pyhanlp.static import download, remove_file, HANLP_DATA_PATH
from pyhanlp import SafeJClass

In [2]:
def test_data_path():
    """
    获取测试数据路径，位于$root/data/test，根目录由配置文件指定。
    :return:
    """
    data_path = os.path.join(HANLP_DATA_PATH, 'test')
    if not os.path.isdir(data_path):
        os.mkdir(data_path)
    return data_path


def ensure_data(data_name, data_url):
    root_path = test_data_path()
    dest_path = os.path.join(root_path, data_name)
    if os.path.exists(dest_path):
        return dest_path
    if data_url.endswith('.zip'):
        dest_path += '.zip'
    download(data_url, dest_path)
    if data_url.endswith('.zip'):
        with zipfile.ZipFile(dest_path, "r") as archive:
            archive.extractall(root_path)
        remove_file(dest_path)
        dest_path = dest_path[:-len('.zip')]
    return dest_path

In [3]:
NaiveBayesClassifier = SafeJClass('com.hankcs.hanlp.classification.classifiers.NaiveBayesClassifier')
IOUtil = SafeJClass('com.hankcs.hanlp.corpus.io.IOUtil')


def train_or_load_classifier():
    corpus_path = ensure_data('搜狗文本分类语料库迷你版',
                              'http://hanlp.linrunsoft.com/release/corpus/sogou-text-classification-corpus-mini.zip')
    model_path = corpus_path + '.ser'
    if os.path.isfile(model_path):
        return NaiveBayesClassifier(IOUtil.readObjectFrom(model_path))
    classifier = NaiveBayesClassifier()
    classifier.train(corpus_path)
    model = classifier.getModel()
    IOUtil.saveObjectTo(model, model_path)
    return model


def predict(classifier, text):
    print("《%16s》\t属于分类\t【%s】" % (text, classifier.classify(text)))
    # 如需获取离散型随机变量的分布，请使用predict接口
    # print("《%16s》\t属于分类\t【%s】" % (text, classifier.predict(text)))

In [4]:
classifier = train_or_load_classifier()
predict(classifier, "C罗压梅西内马尔蝉联金球奖 2017=C罗年")
predict(classifier, "英国造航母耗时8年仍未服役 被中国速度远远甩在身后")
predict(classifier, "研究生考录模式亟待进一步专业化")
predict(classifier, "如果真想用食物解压,建议可以食用燕麦")
predict(classifier, "通用及其部分竞争对手目前正在考虑解决库存问题")

《C罗压梅西内马尔蝉联金球奖 2017=C罗年》	属于分类	【体育】
《英国造航母耗时8年仍未服役 被中国速度远远甩在身后》	属于分类	【军事】
《 研究生考录模式亟待进一步专业化》	属于分类	【教育】
《如果真想用食物解压,建议可以食用燕麦》	属于分类	【健康】
《通用及其部分竞争对手目前正在考虑解决库存问题》	属于分类	【汽车】
